## RGB to GrayScale
(Ground Truth dosyalarında önceden belirlenmiş rgb renklerine göre sadece car, rider ve human görüntülerinin siyah-beyaz şeklinde maskeleme işleminin yapılması)

In [ ]:
# maskelerin hepsini beyaz tona dönüştürme
import os
import numpy as np
from PIL import Image

def rgb_to_single_channel_mask(rgb_mask):
    height, width, _ = rgb_mask.shape
    single_channel_mask = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            pixel = rgb_mask[i, j, :]
            if np.array_equal(pixel, [0, 0, 142]):  # Car
                single_channel_mask[i, j] = 255  # Grayscale value for car
            elif np.array_equal(pixel, [255, 0, 0]):  # Rider
                single_channel_mask[i, j] = 255  # Grayscale value for rider
            elif np.array_equal(pixel, [220, 20, 60]):  # Human
                single_channel_mask[i, j] = 255  # Grayscale value for human

    return single_channel_mask

def process_images(input_dir, output_dir):

    for idx, filename in enumerate(os.listdir(input_dir)):
        if filename.endswith(".png"):

            input_path = os.path.join(input_dir, filename)

            # Load the RGB mask
            rgb_mask = np.array(Image.open(input_path))

            # Convert RGB mask to single-channel mask
            result_mask = rgb_to_single_channel_mask(rgb_mask)

            # Construct the full path for the output mask
            output_path = os.path.join(output_dir, f"{filename}")

            # Save the result mask
            Image.fromarray(result_mask).save(output_path)

            # Print the index of the current image
            print(f"Processed image {idx + 1}/{len(os.listdir(input_dir))}")

# Input and output directories
input_directory = "/content/drive/MyDrive/visea son/Ground Truth"
output_directory = "/content/drive/MyDrive/visea son/7Test the mask"

# Process images and save the converted masks
process_images(input_directory, output_directory)


Processed image 1/25
Processed image 2/25
Processed image 3/25
Processed image 4/25
Processed image 5/25
Processed image 6/25
Processed image 7/25
Processed image 8/25
Processed image 9/25
Processed image 10/25
Processed image 11/25
Processed image 12/25
Processed image 13/25
Processed image 14/25
Processed image 15/25
Processed image 16/25
Processed image 17/25
Processed image 18/25
Processed image 19/25
Processed image 20/25
Processed image 21/25
Processed image 22/25
Processed image 23/25
Processed image 24/25
Processed image 25/25


In [ ]:
import os
import cv2

input_dir = '/content/drive/MyDrive/visea son/7Test the mask'
output_dir = '/content/drive/MyDrive/test label'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)

    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # poligona dönüştürülüyor
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # txt dosyasına poligon şeklinde işleniyor
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

## Mask-to-YoloTxt
(siyah-beyaz maskeler, Yolov8'de segmentasyon aşamasında kullanılmak üzere txt formuna dönüşüm yapılmaktadır.)

In [ ]:
from PIL import Image, ImageDraw

def draw_rectangles(image_path, txt_path, output_path=None):
    # Resmi yükle
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Koordinatları txt dosyasından oku
    with open(txt_path, 'r') as file:
        lines = file.readlines()

    # Her bir satırdaki dikdörtgen koordinatlarını çiz
    for line in lines:
        values = list(map(float, line.split()))
        class_label, *coordinates = values

        num_coordinates = len(coordinates)
        if num_coordinates % 2 == 0 and num_coordinates >= 4:
            # Her çift sırada (x, y) koordinatları
            coordinates = [(coordinates[i], coordinates[i + 1]) for i in range(0, len(coordinates), 2)]

            # Koordinatları resim boyutlarına göre ölçeklendir
            coordinates = [(int(x * image.width), int(y * image.height)) for x, y in coordinates]

            # poigon çiz
            draw.polygon(coordinates, outline=(0, 255, 0), width=2)  # (0, 255, 0) renk: Yeşil, 2 kalınlık

    # Çizilmiş resmi dosyaya kaydet
    if output_path:
        image.save(output_path)
    else:
        image.show()

# Örnek kullanım
image_path = '/content/drive/MyDrive/Visea_KITTI_Dataset/images/train/048.png'  # Resmin yolu
txt_path = '/content/drive/MyDrive/Visea_KITTI_Dataset/labels/train/converted_train_mask_48.txt'  # Poligon koordinatlarının bulunduğu txt dosyasının yolu
#output_path = '/content/output_son.jpg'  # Çizilmiş resmin kaydedileceği dosyanın yolu (opsiyonel)

draw_rectangles(image_path, txt_path, output_path)
